In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

import json
import tensorflowjs as tfjs



## Import Text

In [3]:
# Reading corpus the text file
with open("IndiaUS.txt", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()

## Tokenize Text

In [4]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [5]:
my_input_sequences = []
for line in mytext.split('\n'):
    token_list = mytokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 100)           59900     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 599)               90449     
                                                                 
Total params: 300949 (1.15 MB)
Trainable params: 300949 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
43/43 [==============================] - 5s 78ms/step - loss: 6.2069 - accuracy: 0.0496
Epoch 2/100
43/43 [==============================] - 3s 77ms/step - loss: 5.7969 - accuracy: 0.0576
Epoch 3/100
43/43 [==============================] - 3s 79ms/step - loss: 5.7256 - accuracy: 0.0576
Epoch 4/100
43/43 [==============================] - 3s 80ms/step - loss: 5.6724 - accuracy: 0.0576
Epoch 5/100
43/43 [==============================] - 4s 82ms/step - loss: 5.5903 - accuracy: 0.0576
Epoch 6/100
43/43 [==============================] - 3s 79ms/step - loss: 5.4901 - accuracy: 0.0576
Epoch 7/100
43/43 [==============================] - 3s 78ms/step - loss: 5.3722 - accuracy: 0.0656
Epoch 8/100
43/43 [==============================] - 3s 79ms/step - loss: 5.2231 - accuracy: 0.0832
Epoch 9/100
43/43 [==============================] - 3s 78ms/step - loss: 5.0458 - accuracy: 0.1072
Epoch 10/100
43/43 [==============================] - 3s 80ms/step - loss: 4.8481 - accuracy: 0.1109

## Save Model

In [10]:
tf.keras.models.save_model(model, 'my_model.h5')

C:\Users\lusti\AppData\Local\Temp\ipykernel_17924\676220079.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, 'my_model.h5')


In [20]:

# Zielverzeichnis für das TensorFlow.js-Modell
output_dir = "./tfjs_models/model" 

# Konvertierung des Modells
tfjs.converters.save_keras_model(model, output_dir)


C:\Users\lusti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save Tokenizer

In [12]:
vocab = {}
for word in mytokenizer.word_index:
    vocab[word] = mytokenizer.word_index[word]

with open('vocab.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(vocab, ensure_ascii=False))

In [13]:
# save max_sequence_len to file
with open('max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))


In [19]:
input_text = "Joe biden"
predict_next_words= 6
token_list = mytokenizer.texts_to_sequences([input_text])[0]
print(token_list)
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
print(token_list)

[72, 14]
82
